# WorkFlow

* Intro to Tensors
* Getting info from Tensors
* Manipulating Tensors
* Tensors and Numpy
* Using @tf.function (speed up regular python fucntion)
* Using GPU/TPU w/TensorFlow
* Excercise


### Intro to `Tensors`

In [18]:
# import tensorflow

import tensorflow as tf
print(tf.__version__)

2.11.0


In [19]:
# creating tensors with tf.contant()
scalar = tf.constant(7)
scalar
# has an empty shape
# tf.constant? ceates a constant tensor from a tensor-like object

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [20]:
# check for number of dimensions
scalar.ndim

0

In [21]:
# create a vector
vector = tf.constant([10,10]) # pass a python list to constant function
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [22]:
# check dimension of vector
vector.ndim

1

In [23]:
# create a matrix
matrix = tf.constant([
    [10,7],
    [7,10]
])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [24]:
# check for dimension
matrix.ndim

2

In [25]:
another_matrix = tf.constant([[1.,4.],
[3.,5.],
[9., 12.]], dtype=tf.float16) # reduce data size occuiance on disk

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[ 1.,  4.],
       [ 3.,  5.],
       [ 9., 12.]], dtype=float16)>

In [26]:
another_matrix.ndim #number of elements in a shape

2

In [27]:
# create another tensor
tensor = tf.constant(
    [[[1,2,3,],
        [4,5,6]],
        [[7,8,9],
        [10,11,12]],
        [[13,14,15],
        [16,17,18]]]
)
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [28]:
tensor.ndim

3

### Creating Tensors with `tf.Variable`

In [29]:
changable_tensor = tf.Variable([10,7])
unchangable_tensor = tf.constant([10,7])

changable_tensor, unchangable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [30]:
# lets chnag one of the elements in a chnagable tensor
# changable_tensor[0] = 13

In [31]:
# trying assign
changable_tensor[1].assign(15)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([10, 15], dtype=int32)>

### Creating Random Tensors

- The neural net always initializes itself with random weights/random tensors
- Adjusts the random weights wrt what the net learns

In [32]:
# create two random tensors
random_1 = tf.random.Generator.from_seed(42) # set the seed for reproducibility
random_1 = random_1.normal(shape=(4,2))
random_1

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ],
       [ 0.09988727, -0.50998646]], dtype=float32)>

In [33]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(4,2))
random_2

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ],
       [ 0.09988727, -0.50998646]], dtype=float32)>

In [34]:
# with both having a reproducibility value of 42
random_1, random_1, random_1 == random_2 # check hoe they compare

(<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [35]:
random_2 = tf.random.Generator.from_seed(2) # change the seed value to 2 and see how that compares
random_2 = random_2.normal(shape=(4,2))
random_2

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.1012345 , -0.2744976 ],
       [ 1.4204658 ,  1.2609464 ],
       [-0.43640924, -1.9633987 ],
       [-0.06452483, -1.056841  ]], dtype=float32)>

In [36]:
random_2, random_1, random_2 == random_1 # change the seed value and the reproducibility is different

(<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.1012345 , -0.2744976 ],
        [ 1.4204658 ,  1.2609464 ],
        [-0.43640924, -1.9633987 ],
        [-0.06452483, -1.056841  ]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=bool, numpy=
 array([[False, False],
        [False, False],
        [False, False],
        [False, False]])>)

### Shuffling: 
* Inheret Order shouldn't affect learning

In [37]:
# not_shuffled = tf.random.Generator.from_seed(42)
# not_shuffled = not_shuffled.normal(shape=(3,2))
# not_shuffled

not_shuffled = tf.constant([[1,2],
[3,4],
[5,6]])
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [38]:
shuffled = tf.random.shuffle(not_shuffled)
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [5, 6],
       [3, 4]], dtype=int32)>

In [39]:
not_shuffled, shuffled, not_shuffled == shuffled

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [5, 6],
        [3, 4]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [False, False],
        [False, False]])>)

In [40]:
# global set_seed
tf.random.set_seed(42) # global
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [49]:
# create a tensor array of all zeros
tf.zeros(shape=(2,4), dtype=tf.int32)

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int32)>

In [50]:
tf.ones([4,6], tf.float64)

<tf.Tensor: shape=(4, 6), dtype=float64, numpy=
array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])>

#### Convert a NumPy Array into a Tensor

In [52]:
# import numpy
import numpy as np
numpy_A = np.arange(1,25,dtype=np.int16)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int16)

In [55]:
# convert the Array into a Tensor
tensor_t = tf.constant(numpy_A)
tensor_t
tensor_tshaped = tf.constant(numpy_A, shape=(2,4,3))
tensor_tshaped

<tf.Tensor: shape=(2, 4, 3), dtype=int16, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18],
        [19, 20, 21],
        [22, 23, 24]]], dtype=int16)>

In [59]:
numpy_B = np.arange(12.3,31.1, dtype=np.float16)
numpy_B

array([12.3, 13.3, 14.3, 15.3, 16.3, 17.3, 18.3, 19.3, 20.3, 21.3, 22.3,
       23.3, 24.3, 25.3, 26.3, 27.3, 28.3, 29.3, 30.3], dtype=float16)

In [63]:
tensor_tb = tf.constant(numpy_B, dtype=tf.int16)
tensor_tb

<tf.Tensor: shape=(19,), dtype=int16, numpy=
array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30], dtype=int16)>